In [11]:
import nltk
import json
import spacy
from nltk.corpus import stopwords
from math import log
from collections import defaultdict, Counter
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer

OPEN_QUESTION_WORDS = ['what', 'who', 'whose', 'whom', 'where', 'when', 'why', 'how',
                       'which', "what's", "who's", "where's", "how's"]
CLOSED_QUESTION_WORDS = ['is', 'are', 'am', 'was', 'were', 'do', 'does,', 'did', 'can',
                         'could', 'will', 'would', 'shall', 'should', 'have', 'has',
                         'had']

# Stop words
stop = set(stopwords.words('english'))

lmtz = WordNetLemmatizer()

with open('testing.json') as json_data:
    test = json.load(json_data)

with open('documents.json') as json_data:
    documents = json.load(json_data)

# Spacy toolkit
nlp = spacy.load('en_core_web_sm')

punc = set(punctuation)


def strip_punctuation(s):
    return ''.join(c for c in s if c not in punc)


def lemmatize(token):
    lemma = lmtz.lemmatize(token, 'v')
    if lemma == token:
        lemma = lmtz.lemmatize(token, 'n')
    return lemma


def extract_term_freqs(doc):
    tfs = {}
    for token in nltk.word_tokenize(doc):
        lemma = lemmatize(token.lower())
        if lemma not in stop and lemma.isalpha():
            tfs[lemma] = tfs.get(lemma, 0) + 1
    return tfs


def compute_doc_freqs(doc_term_freqs):
    dfs = Counter()
    for tfs in doc_term_freqs.values():
        for term in tfs.keys():
            dfs[term] += tfs[term]
    return dfs


def query_vsm(query, index, k=10):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator.most_common(k)


# Find the question word
def get_qword(question):
    tokens = nltk.word_tokenize(question.lower())
    for token in tokens:
        if token in OPEN_QUESTION_WORDS:
            return token
    for token in tokens:
        if token in CLOSED_QUESTION_WORDS:
            return token
    return 'others'


file = open('Second_method.csv', 'w')
file.write('id,answer\n')

case_count = 0
# test = [test[17]]
for test_case in test:
    question = test_case['question']
    docid = test_case['docid']

    # Convert doc into one string, then tokenize sentences
    corpus = ''
    for para in documents[docid]['text']:
        corpus += para + ' '

    # sentence as a document
    raw_docs = nltk.sent_tokenize(corpus)

    # TFIDF
    doc_term_freqs = {}
    for (id, raw_doc) in enumerate(raw_docs):
        term_freqs = extract_term_freqs(raw_doc)
        doc_term_freqs[id] = term_freqs
    M = len(doc_term_freqs)

    doc_freqs = compute_doc_freqs(doc_term_freqs)

    vsm_inverted_index = defaultdict(list)
    for docid, term_freqs in doc_term_freqs.items():
        N = sum(term_freqs.values())
        length = 0

        # find tf*idf values and accumulate sum of squares
        tfidf_values = []
        for term, count in term_freqs.items():
            tfidf = float(count) / N * log(M / float(doc_freqs[term]))
            tfidf_values.append((term, tfidf))
            length += tfidf ** 2

        # normalise documents by length and insert into index
        length = length ** 0.5
        for term, tfidf in tfidf_values:
            # inversion of the indexing, term -> (doc_id, score)
            vsm_inverted_index[term].append([docid, tfidf / length])

    for term, docids in vsm_inverted_index.items():
        docids.sort()

    terms = extract_term_freqs(question)
    results = query_vsm(terms, vsm_inverted_index)

    # Step 2
    # Analyse question type
    qword = get_qword(question)

    # the word after question word, such as 'what value', 'which gender'
    next_token = ''

    target = [] # target dep

    # dependency parsing
    dep = ''

    # head word
    head = ''

    # head dependency
    head_dep = ''

    # subject, root, object
    nsubj = ''
    ROOT = ''
    dobj = ''
    attr = ''

    # yes or no questions have two options
    closed_q_choices = ('', '')

    doc = nlp(question)

    tokens = nltk.word_tokenize(question.lower())

    # get next word
    if qword in tokens:
        if tokens.index(qword) < len(tokens) - 1:
            next_token = tokens[tokens.index(qword) + 1]

    # get structure of question
    for token in doc:
        if 'nsubj' in token.dep_ and token.head.dep_ == 'ROOT':
            nsubj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'ROOT':
            ROOT = lemmatize(strip_punctuation(token.text))
        if 'dobj' in token.dep_ and token.head.dep_ == 'ROOT':
            dobj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'attr' and token.head.dep_ == 'ROOT':
            attr = lemmatize(strip_punctuation(token.text))

    # determine answer dependency
    if qword in ['who', "who's", 'whom', 'whose']:
        for chunk in doc.noun_chunks:
            if qword in chunk.text:
                target = [chunk.root.dep_]

    elif qword in ['where', "where's", 'when']:
        target = ['advmod','prep']

    elif qword in ['how', "how's"]:
        if next_token in ['much', 'many','long','far', 'big', 'wide', 'deep', 'tall', 'high', 'fast', 'heavy','old', 'young']:
            target = ['attr']
        
        elif next_token in ['does', 'did', 'do', 'have', 'has', 'had', 'should',
                            'can', 'could', 'will', 'would', 'must']:
            if dobj != '':
                target = ['advmod', 'acomp']
            else:
                target = ['ROOT']

    elif qword in ['what', "what's", 'which']:
        # what...do type question
        tokens.remove(next_token)
        if 'do' in tokens:
            target = ['ROOT']
        else:
            for chunk in doc.noun_chunks:
                if qword in chunk.text:
                    target = [chunk.root.dep_]

    elif qword == 'why':
        target = ['mark','prep']

    elif qword in CLOSED_QUESTION_WORDS:
        # answer is one of the 'or' options in the question
        if 'or' in tokens:
            index = tokens.index('or')
            
            # dep of phrase before and after 'or'
            prev = tokens[index - 1]
            for chunk in doc.noun_chunks:
                if prev in chunk.text:
                    target = [chunk.root.dep_]
            
            if target == '':
                for token in doc:
                    if prev in token.text:
                        target = [token.dep_]

    # find answer with highest score from the 10 most similar sentences
    max_score = -1
    answer = ''

    for id, _ in results:
        sent = raw_docs[id]
        doc = nlp(sent)

        # find sentence structure
        sent_nsubj = ''
        sent_ROOT = ''
        sent_dobj = ''
        sent_attr = ''
        for token in doc:
            if 'nsubj' in token.dep_ and token.head.dep_ == 'ROOT':
                sent_nsubj = lemmatize(strip_punctuation(token.text))
            if token.dep_ == 'ROOT':
                sent_ROOT = lemmatize(strip_punctuation(token.text))
            if 'dobj' in token.dep_ and token.head.dep_ == 'ROOT':
                sent_dobj = lemmatize(strip_punctuation(token.text))
            if 'attr' in token.dep_ and token.head.dep_ == 'ROOT':
                sent_attr = lemmatize(strip_punctuation(token.text))

        score = 0
        if nsubj == sent_nsubj:
            score += 1
            if ROOT == sent_ROOT:
                score += 1
                if dobj == sent_dobj:
                    score += 1
                if attr == sent_attr:
                    score += 1

        max_score = -1
        answer = ''
        if target == ['ROOT'] or target == ['attr'] or target == ['advmod','acomp']:
            for token in doc:
                if token.dep_ in target:
                    score += 1
                    
                    if token.text not in question:
                        score += 1
                        
                    if strip_punctuation(token.text).lower() in stop:
                        score = -1
                        
                    if token.text.lower() == 'it':
                        score = -1
                    
                    if score > max_score:
                        max_score = score
                        answer = token.text
                        
        elif target == ['mark','prep'] or target == ['advmod','prep']:
            for token in doc:
                if token.dep_ in target:
                    score += 1
                    
                    if token.text not in question:
                        score += 1
                        
                    if strip_punctuation(token.text).lower() in stop:
                        score = -1
                        
                    if token.text.lower() == 'it':
                        score = -1
                    
                    if score > max_score:
                        max_score = score
                        substr = sent[sent.index(token.text):]
                        tokens = nltk.word_tokenize(substr)
                        del tokens[0]
                        for token in tokens:
                            if token in punc:
                                break
                            answer += token+' '
                        
        else:
            for chunk in doc.noun_chunks:
                if chunk.root.dep_ in target:
                    score += 1
                    
                    if chunk.text not in question:
                        score += 1
                        
                    if strip_punctuation(chunk.text).lower() in stop:
                        score = -1
                        
                    if chunk.text.lower() == 'it':
                        score = -1
                    
                    if score > max_score:
                        max_score = score
                        answer = chunk.text
        
    # default answer
    if answer == '':
        for chunk in doc.noun_chunks:
            if chunk.text not in stop and chunk.text not in question:
                answer = chunk.text
        
        
    file.write(str(case_count))
    file.write(',')
    file.write(strip_punctuation(answer).strip().lower())
    file.write('\n')
    print(case_count, ' ', answer)
    case_count += 1

file.close()

0   all capabilities
1   the World Wide Web Foundation
2   the user
3   the open source software model
4   started his own company 
5   month
6   referred to as a browser 
7   Java applets
8   Internet Explorer
9   the World Wide Web
10   part
11   Sir Tim Berners-Lee
12   merely "web browsers
13   a web browser
14   merely "web browsers
15   started his own company 
16   Commission Safari and Mobile Safari were likewise always included with OS X and iOS respectively always included with OS X and iOS respectively included with OS X and iOS respectively 
17   local files
18   a process
19   history
20   a web browser
21   released in September 2008 
22   Net Applications
23   the World Wide Web
24   over 95%
25   e-mail
26   reduces consumer choice 
27   their managers 
28   web browsers
29   (HTTP
30   CSS
31   the browser
32   web browsers
33   the World Wide Web Foundation
34   users
35   over 40 million phones
36   the average person
37   Internet Explorer
38   a web browser
39   Mi

337   more than the number of protesters 
338   the United States
339   their own "Human Rights Torch
340   all times
341   the Chinese government's crackdown
342   protests
343   Tibet
344   requested the torch route in San Francisco be shortened 
345   Jinnah Stadium
346   August
347   the Chinese media
348   a Beijing citizen
349   to Asian Times 
350   the curtailed 3 km route 
351   propane
352   its own face
353   a sound
354   slower oscillation
355   C♯
356   frequency
357   the waveform
358   the number
359   frequency
360   higher frequencies
361   fundamental frequency
362   upper partials
363   the frequencies
364   the pitches
365   the basic prerequisite
366   higher frequencies
367   fundamental frequency
368   a pitch
369   any specific pitch
370   several permanent and travelling exhibitions
371   the River Itchen
372   apparent crime figures
373   ruins
374   local public transport
375   jobs
376   England
377   Carnival UK
378   boys’ and girls’ teams
379   elections

702   cruise operator Carnival UK
703   the city's buildings
704   the base for any cross-channel ferries 
705   the River Itchen
706   dance music
707   the town
708   Southampton Town
709   England
710   the city's waterfront
711   the town
712   the friars
713   Ronald Koeman
714   Southampton Central
715   an ethnic group
716   this reference point
717   association
718   Jews
719   local Christian rulers
720   Ashkenazi maternal lineages
721   the Caucasus region
722   3 million DNA letters
723   the general European population
724   outnumbered Ashkenazim three to two '' 
725   the pre-2006 origin hypothesis
726   the Bray et al
727   the second half
728   recent times
729   subgroup
730   their fellow non-Jewish countrymen
731   Israelis
732   other parts
733   the Bray et al
734   together
735   differing rates
736   the Roman empire
737   the practice
738   a nun נ
739   refers to the nusach Ashkenaz 
740   the Khazar hypothesis
741   Orthodox Judaism
742   faiths
743   the la

1019   the Theotokos
1020   Constantinople
1021   Georgia
1022   Roman prototypes
1023   strong Italianate influence
1024   the Theotokos
1025   The mosaics
1026   a beautiful mosaic pavement
1027   three medallions
1028   a Samaritan synagogue
1029   initiated in these arts '' – says the chronicler about the role of the Greeks in the revival of mosaic art in medieval Italy 
1030   Kiev
1031   The most superb example
1032   fully coloured work
1033   slaves
1034   the Lusosphere
1035   the city
1036   Hungary
1037   a robot
1038   Empress Zoe
1039   the 8th century
1040   grains
1041   Cipriana
1042   bland mosaics
1043   figures
1044   the Dome
1045   the Deacon
1046   
1047   , lesser known sites
1048   Georgia
1049   initiated in these arts '' – says the chronicler about the role of the Greeks in the revival of mosaic art in medieval Italy 
1050   the 16th century
1051   Thessaloniki
1052   the church
1053   Bethlehem
1054   the 8th century
1055   only the floor panels
1056   mosaic

1311   Eastern Orthodoxy
1312   batteries
1313   the Crimean peninsula
1314   disaster
1315   a Franco-British naval force
1316   one American officer
1317   orders
1318   equipment
1319   30 March
1320   the Greek army
1321   part
1322   the more sweeping demands
1323   Sinop
1324   the nearest future
1325   his protection
1326   equipment
1327   the Crimean Peninsula
1328   the Crimean peninsula
1329   Charlemagne
1330   the previous war
1331   Alger
1332   Alfred Nobel
1333   ordered
1334   a German satellite
1335   Rear Admiral Yevfimy Putyatin
1336   problems to the south and the heavy artillery was brought ashore with batteries and connecting trenches built so that by 10 October some batteries were ready and by 17 October—when the bombardment commenced—126 guns were firing 
1337   the day
1338   direct conflict
1339   Oltenița
1340   Italy
1341   the attack
1342   27/28 October
1343   Gallipoli
1344   Russia
1345   the Russians
1346   the French Emperor
1347   William Howard Russ

...

1986   KU Med
1987   the three campuses
1988   "Bleeding Kansas
1989   the first college building
1990   seven locations
1991   Lawrence, Kansas
1992   university
1993   the Kansas City metropolitan area
1994   three NCAA tournament championships in 1952 
1995   accounting
1996   urban planning
1997   accounting
1998   university
1999   any other university
2000   Kansas City
2001   "Bleeding Kansas
2002   a public university
2003   the 2014 U.S. News & World Report "Best Graduate Schools" edition
2004   Kansas
2005   accounting
2006   includes KU Dining Services the Kansas Union 
2007   graduate study
2008   Student Union Activities
2009   additional fees
2010   any other university
2011   accounting
2012   295
2013   Planning
2014   Social Welfare
2015   of the university 's William Allen White School of Journalism and reaches an audience of at least 30,000 daily readers through its print and online publications The university houses the following public broadcasting stations 
2016  

2302   traditional racial groupings
2303   some European ancestors
2304   their descent groups
2305   biological races
2306   such [racial] theories
2307   Homo
2308   Biological Races
2309   other such means
2310   groups
2311   various human groups
2312   recent times
2313   each race/ethnicity group
2314   population
2315   haplogroup
2316   the limits
2317   race
2318   social categorization
2319   European Y-DNA
2320   their descent groups
2321   intellectual, behavioral, and moral qualities
2322   Homo
2323   such [racial] theories
2324   East Asian sweeps
2325   individual phenotypes
2326   European Y-DNA
2327   the variation
2328   traditional racial groupings
2329   nineteen rejected race
2330   this African population
2331   the emigrating groups
2332   this new color-blind racism
2333   "Indian blood
2334   social constructs
2335   each group
2336   New Guinea
2337   the popular imagination
2338   minor ones
2339   intellectual, behavioral, and moral qualities
2340   this Af

2636   passengers
2637   the local jurisdiction
2638   The first elevator shaft
2639   the most efficient design
2640   the ASME A17.2 Standard
2641   the ASME A17.2 Standard
2642   machine room
2643   the lower city
2644   the car
2645   a plunger gripping device
2646   the car
2647   Wembley
2648   West Germany
2649   their first and only World Cup title
2650   two years
2651   the final
2652   their first and only World Cup title
2653   the away shirt
2654   King Richard I
2655   the 2006 FIFA World Cup's conclusion
2656   their first and only World Cup title
2657   at least one FIFA World Cup title
2658   the away shirt
2659   The team
2660   the World Cup
2661   West Germany
2662   the 2006 FIFA World Cup
2663   Central Europe
2664   West Germany
2665   third place
2666   Ramsey's dismissal
2667   central and northern Scotland
2668   the first round
2669   King Richard I
2670   Central Europe
2671   West Germany
2672   that company's collapse
2673   King Richard I
2674   the 2006 

2962   battery life
2963   the memory issue
2964   Apple
2965   the more expensive Macintosh systems
2966   a dealer
2967   other personal computers
2968   keyboard
2969   Claris Impact
2970   potential buyers
2971   second
2972   the iMac G4
2973   the problem
2974   larger monitors
2975   the average consumer
2976   a whole
2977   security updates
2978   from its fast 40 MHz 68030 processor 
2979   hard drives
2980   later appeals
2981   version
2982   nine months
2983   Snow Leopard
2984   VirtualBox
2985   iteration
2986   a price
2987   Microsoft's formerly separate MS-DOS and Windows products
2988   1998 phased out the Macintosh name in favor of `` Mac '' 
2989   Microsoft Windows
2990   laptops
2991   the Legislative Assembly
2992   cable
2993   the third-fastest growing economy
2994   its difficult terrain
2995   marketing facilities
2996   Pathankot–Jogindernagar
2997   various other house hold items
2998   in the list of the highest per capita incomes of Indian states 
2999  

3295   the underside
3296   plants
3297   use suckers at their front and rear ends to move like inchworms 
3298   ammonia
3299   trademarks
3300   the general and permanent federal statutes
3301   wire fraud
3302   the Supreme Court
3303   the federal judiciary
3304   contemporary English cases
3305   American reports
3306   the Constitution
3307   criminal law
3308   the rest
3309   precedents
3310   given
3311   stare decisis
3312   the territories
3313   the Constitution
3314   mail
3315   mail
3316   American tort law
3317   the Constitution
3318   the territories
3319   federal agencies
3320   on 
3321   law enforcement powers
3322   abolished
3323   state courts
3324   the Constitution
3325   mail
3326   the Uniform Commercial Code
3327   criminal law
3328   272,795 new cases
3329   stare decisis
3330   issue
3331   a settlement
3332   post-Revolution Commonwealth rulings
3333   1.9 million juvenile cases
3334   state courts
3335   West Publishing
3336   several ways
3337   sever

# Train set test

In [15]:
import nltk
import json
import spacy
from nltk.corpus import stopwords
from math import log
from collections import defaultdict, Counter
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer

OPEN_QUESTION_WORDS = ['what', 'who', 'whose', 'whom', 'where', 'when', 'why', 'how',
                       'which', "what's", "who's", "where's", "how's"]
CLOSED_QUESTION_WORDS = ['is', 'are', 'am', 'was', 'were', 'do', 'does,', 'did', 'can',
                         'could', 'will', 'would', 'shall', 'should', 'have', 'has',
                         'had']

# Stop words
stop = set(stopwords.words('english'))

lmtz = WordNetLemmatizer()

with open('testing.json') as json_data:
    test = json.load(json_data)

with open('training.json') as json_data:
    train = json.load(json_data)
    
with open('documents.json') as json_data:
    documents = json.load(json_data)

# Spacy toolkit
nlp = spacy.load('en_core_web_sm')

punc = set(punctuation)


def strip_punctuation(s):
    return ''.join(c for c in s if c not in punc)


def lemmatize(token):
    lemma = lmtz.lemmatize(token, 'v')
    if lemma == token:
        lemma = lmtz.lemmatize(token, 'n')
    return lemma


def extract_term_freqs(doc):
    tfs = {}
    for token in nltk.word_tokenize(doc):
        lemma = lemmatize(token.lower())
        if lemma not in stop and lemma.isalpha():
            tfs[lemma] = tfs.get(lemma, 0) + 1
    return tfs


def compute_doc_freqs(doc_term_freqs):
    dfs = Counter()
    for tfs in doc_term_freqs.values():
        for term in tfs.keys():
            dfs[term] += tfs[term]
    return dfs


def query_vsm(query, index, k=10):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator.most_common(k)


# Find the question word
def get_qword(question):
    tokens = nltk.word_tokenize(question.lower())
    for token in tokens:
        if token in OPEN_QUESTION_WORDS:
            return token
    for token in tokens:
        if token in CLOSED_QUESTION_WORDS:
            return token
    return 'others'


# file = open('Second_method.csv', 'w')
# file.write('id,answer\n')

case_count = 0
# test = [test[17]]
for train_case in train:
    question = train_case['question']
    docid = train_case['docid']
    correct_answer = train_case['text']

    # Convert doc into one string, then tokenize sentences
    corpus = ''
    for para in documents[docid]['text']:
        corpus += para + ' '

    # sentence as a document
    raw_docs = nltk.sent_tokenize(corpus)

    # TFIDF
    doc_term_freqs = {}
    for (id, raw_doc) in enumerate(raw_docs):
        term_freqs = extract_term_freqs(raw_doc)
        doc_term_freqs[id] = term_freqs
    M = len(doc_term_freqs)

    doc_freqs = compute_doc_freqs(doc_term_freqs)

    vsm_inverted_index = defaultdict(list)
    for docid, term_freqs in doc_term_freqs.items():
        N = sum(term_freqs.values())
        length = 0

        # find tf*idf values and accumulate sum of squares
        tfidf_values = []
        for term, count in term_freqs.items():
            tfidf = float(count) / N * log(M / float(doc_freqs[term]))
            tfidf_values.append((term, tfidf))
            length += tfidf ** 2

        # normalise documents by length and insert into index
        length = length ** 0.5
        for term, tfidf in tfidf_values:
            # inversion of the indexing, term -> (doc_id, score)
            vsm_inverted_index[term].append([docid, tfidf / length])

    for term, docids in vsm_inverted_index.items():
        docids.sort()

    terms = extract_term_freqs(question)
    results = query_vsm(terms, vsm_inverted_index)

    # Step 2
    # Analyse question type
    qword = get_qword(question)

    # the word after question word, such as 'what value', 'which gender'
    next_token = ''

    target = [] # target dep

    # dependency parsing
    dep = ''

    # head word
    head = ''

    # head dependency
    head_dep = ''

    # subject, root, object
    nsubj = ''
    ROOT = ''
    dobj = ''
    attr = ''

    # yes or no questions have two options
    closed_q_choices = ('', '')

    doc = nlp(question)

    tokens = nltk.word_tokenize(question.lower())

    # get next word
    if qword in tokens:
        if tokens.index(qword) < len(tokens) - 1:
            next_token = tokens[tokens.index(qword) + 1]

    # get structure of question
    for token in doc:
        if 'nsubj' in token.dep_ and token.head.dep_ == 'ROOT':
            nsubj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'ROOT':
            ROOT = lemmatize(strip_punctuation(token.text))
        if 'dobj' in token.dep_ and token.head.dep_ == 'ROOT':
            dobj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'attr' and token.head.dep_ == 'ROOT':
            attr = lemmatize(strip_punctuation(token.text))

    # determine answer dependency
    if qword in ['who', "who's", 'whom', 'whose']:
        for chunk in doc.noun_chunks:
            if qword in chunk.text:
                target = [chunk.root.dep_]

    elif qword in ['where', "where's", 'when']:
        target = ['advmod','prep']

    elif qword in ['how', "how's"]:
        if next_token in ['much', 'many','long','far', 'big', 'wide', 'deep', 'tall', 'high', 'fast', 'heavy','old', 'young']:
            target = ['attr']
        
        elif next_token in ['does', 'did', 'do', 'have', 'has', 'had', 'should',
                            'can', 'could', 'will', 'would', 'must']:
            if dobj != '':
                target = ['advmod', 'acomp']
            else:
                target = ['ROOT']

    elif qword in ['what', "what's", 'which']:
        # what...do type question
        tokens.remove(next_token)
        if 'do' in tokens:
            target = ['ROOT']
        else:
            for chunk in doc.noun_chunks:
                if qword in chunk.text:
                    target = [chunk.root.dep_]

    elif qword == 'why':
        target = ['mark','prep']

    elif qword in CLOSED_QUESTION_WORDS:
        # answer is one of the 'or' options in the question
        if 'or' in tokens:
            index = tokens.index('or')
            
            # dep of phrase before and after 'or'
            prev = tokens[index - 1]
            for chunk in doc.noun_chunks:
                if prev in chunk.text:
                    target = [chunk.root.dep_]
            
            if target == '':
                for token in doc:
                    if prev in token.text:
                        target = [token.dep_]

    # find answer with highest score from the 10 most similar sentences
    max_score = -1
    answer = ''

    for id, _ in results:
        sent = raw_docs[id]
        doc = nlp(sent)

        # find sentence structure
        sent_nsubj = ''
        sent_ROOT = ''
        sent_dobj = ''
        sent_attr = ''
        for token in doc:
            if 'nsubj' in token.dep_ and token.head.dep_ == 'ROOT':
                sent_nsubj = lemmatize(strip_punctuation(token.text))
            if token.dep_ == 'ROOT':
                sent_ROOT = lemmatize(strip_punctuation(token.text))
            if 'dobj' in token.dep_ and token.head.dep_ == 'ROOT':
                sent_dobj = lemmatize(strip_punctuation(token.text))
            if 'attr' in token.dep_ and token.head.dep_ == 'ROOT':
                sent_attr = lemmatize(strip_punctuation(token.text))

        score = 0
        if nsubj == sent_nsubj:
            score += 1
            if ROOT == sent_ROOT:
                score += 1
                if dobj == sent_dobj:
                    score += 1
                if attr == sent_attr:
                    score += 1

        max_score = -1
        answer = ''
        if target == ['ROOT'] or target == ['attr'] or target == ['advmod','acomp']:
            for token in doc:
                if token.dep_ in target:
                    score += 1
                    
                    if token.text not in question:
                        score += 1
                        
                    if strip_punctuation(token.text).lower() in stop:
                        score = -1
                        
                    if token.text.lower() == 'it':
                        score = -1
                    
                    if score > max_score:
                        max_score = score
                        answer = token.text
                        
        elif target == ['mark','prep'] or target == ['advmod','prep']:
            for token in doc:
                if token.dep_ in target:
                    score += 1
                    
                    if token.text not in question:
                        score += 1
                        
                    if strip_punctuation(token.text).lower() in stop:
                        score = -1
                        
                    if token.text.lower() == 'it':
                        score = -1
                    
                    if score > max_score:
                        max_score = score
                        substr = sent[sent.index(token.text):]
                        tokens = nltk.word_tokenize(substr)
                        del tokens[0]
                        for token in tokens:
                            if token in punc:
                                break
                            answer += token+' '
                        
        else:
            for chunk in doc.noun_chunks:
                if chunk.root.dep_ in target:
                    score += 1
                    
                    if chunk.text not in question:
                        score += 1
                        
                    if strip_punctuation(chunk.text).lower() in stop:
                        score = -1
                        
                    if chunk.text.lower() == 'it':
                        score = -1
                    
                    if score > max_score:
                        max_score = score
                        answer = chunk.text
        
    # default answer
    if answer == '':
        for chunk in doc.noun_chunks:
            if chunk.text not in stop and chunk.text not in question:
                answer = chunk.text
        
        
#     file.write(str(case_count))
#     file.write(',')
#     file.write(strip_punctuation(answer).strip().lower())
#     file.write('\n')
    if answer != correct_answer:
        print('Correct: ',correct_answer)
    
        print('Predicted: ',answer)
       
        print('Question: ',question)
        
        for id, _ in results:
            sent = raw_docs[id]
            doc = nlp(sent)
            
        print('\n')
    case_count += 1


Correct:  6966662606895999999♠6.62606896×10−34 j⋅s
Predicted:  more detailed information
Question:  A kilogram could be definined as having a Planck constant of what value?


Correct:  cylinder
Predicted:  diagram
Question:  What is the shape of the object that establishes the base unit of the kilogram?


Correct:  time vs. energy
Predicted:  a given time
Question:  What example is given as another paired relationship of uncertainly related to standard deviation?


Correct:  quantum of action
Predicted:  angular momentum
Question:  What does the Planck Constant refer to?


Correct:  1913
Predicted:  calculated in the early twentieth century 
Question:  When was the first quantized model of the atom introduced?


Correct:  kirchhoff
Predicted:  J⋅s
Question:  What scientist first studied black body radiation?


Correct:  schrödinger
Predicted:  the Bohr model
Question:  Who helped to give the correct quantization rules for electrons in 1926?


Correct:  h
Predicted:  Rutherford's classi

Correct:  the rayleigh–jeans
Predicted:  a continuous variable
Question:  What rule predicted narrow range of energy values at lower temperatures?


Correct:  platinum–iridium
Predicted:  conventional electrical units
Question:  What alloy is the base unit of the kilogram made from?


Correct:  blackbody emissions
Predicted:  the new definition
Question:  What does Planck's law correctly predict?


Correct:  niels bohr
Predicted:  angular momentum
Question:  Who modeled the atom in 1913, challenging Rutherford's model?


Correct:  2007
Predicted:  present
Question:  The value quoted here for the Planck constant is based on a measurement in what year?


Correct:  thermal light emission
Predicted:  the colour
Question:  What is maximized as a result of a black object absorbing all the light that hits it?


Correct:  6981358000000000000♠3.58×10−19 j
Predicted:  a certain kinetic energy
Question:  What is the energy of a photon?


Correct:  white hot
Predicted:  hot atoms
Question:  What c

Correct:  lester brown
Predicted:  the RES Directive
Question:  Who believes that the market does not value nature's service adequately?


Correct:  144
Predicted:  plants
Question:  How many countries now have renewable energy policies?


Correct:  19 percent
Predicted:  an indicative renewable energy target
Question:  Renewables contributed what percentage to our energy consumption?


Correct:  dedicated energy crops
Predicted:  many countries
Question:  What is a promising cellulose source that can be sustainably produced in many regions?


Correct:  coal
Predicted:  many countries
Question:  Some countries have phased out all subsidies for what substance?


Correct:  the internet
Predicted:  the renewable energy and energy efficiency industries
Question:  What was the result of publicly funded links among computers in government labs and reserach institutes?


Correct:  2011
Predicted:  conventional energy sources
Question:  As of what year have there been substantial reductions in

Correct:  brussels , belgium
Predicted:  the Indiana Jones films
Question:  Where did Adventures of Tintin debut?


Correct:  co-producer
Predicted:  Winfrey
Question:  What was Spielberg's role on 'Batteries Not Included'?


Correct:  five
Predicted:  the first Monkey Island games
Question:  How many Oscar nominations did 'Munich' get?


Correct:  26-minute
Predicted:  Broadway
Question:  How long was 'Amblin'?


Correct:  paramount pictures
Predicted:  the editing department
Question:  Which studio produced 'Super 8'?


Correct:  actress
Predicted:  their relationship
Question:  What was Spielberg's future wife's career?


Correct:  four
Predicted:  a major Hollywood studio
Question:  How many TV movies was Spielberg signed to direct?


Correct:  allen daviau
Predicted:  the Third Kind
Question:  Which childhood friend worked on Spielberg's films?


Correct:  brian de palma
Predicted:  the film
Question:  Who introduced Irving to Spielberg?


Correct:  sean connery
Predicted:  the In

Correct:  oskar schindler
Predicted:  the Normandy landing
Question:  Whose life was 'Schindler's List' based on?


Correct:  jaws
Predicted:  the Third Kind
Question:  What movie was Spielberg working on when he first played Pong?


Correct:  two
Predicted:  his direction
Question:  How many Oscars did Close Encounters win?


Correct:  richmond , virginia
Predicted:  never worked with the same screenwriter in his films 
Question:  Where was 'Lincoln' filmed?


Correct:  1975
Predicted:  the water scenes in Jaws are filmed from the low-angle perspective of someone swimming 
Question:  When was Jaws released?


Correct:  `` bachelor funky ''
Predicted:  a movie-making simulation game
Question:  How did Irving describe Spielberg's house?


Correct:  firelight
Predicted:  an actress
Question:  What film led to Close Encounters?


Correct:  capt . miller
Predicted:  that end scene
Question:  Who did Tom Hanks play in 'Saving Private Ryan'?


Correct:  halo
Predicted:  Martin Landau
Questio

...

Correct:  experimental
Predicted:  BYU No
Question:  What type of theater is Nelke Theatre primarily used for?


Correct:  88
Predicted:  study
Question:  How many freshman at BYU were Merit Scholars in 2006?


Correct:  19
Predicted:  the school's mascot
Question:  At what age, since 2012, are women allowed to serve a mission after high school graduation?


Correct:  art rascon
Predicted:  The Ohio State University
Question:  Which former CBS News correspondent graduated from BYU?


Correct:  three
Predicted:  one
Question:  How many television production studios is BYU Broadcasting Technical Operations Center home to?


Correct:  98
Predicted:  Merrill J. Bateman
Question:  How many miles of shelving are in BYU's Harold B. Lee Library?


Correct:  university of deseret
Predicted:  their fields
Question:  What college did BYU separate from to become its own entity?


Correct:  68
Predicted:  these programs
Question:  How many master's programs does BYU have?


Correct:  byu recycles
P

Correct:  culturally conservative
Predicted:  its US students
Question:  What BYU environment seems to surprise many visitors to BYU as well as to the Utah Valley?


Correct:  college of engineering
Predicted:  an independent
Question:  Which BYU college was founded by former alumnus Harvey Fletcher?


Correct:  warren dusenberry
Predicted:  two No
Question:  Who began the school that previously existed at the site where BYU is now located?


Correct:  tracy hall
Predicted:  the rating
Question:  Which notable former BYU student invented the man-made diamond?


Correct:  over three quarters
Predicted:  their apartments
Question:  What percentage of the student body of BYU has some proficiency in a second language?


Correct:  brigham young
Predicted:  the current campus
Question:  Which president of the LDS church purchased the Lewis Building after hinting at the building of a school?


Correct:  22
Predicted:  the previous policy
Question:  What is the average age that BYU students ma

Correct:  progressive-scan
Predicted:  NBC owned and operated station WRC-TV
Question:  Most computer monitors operate in what mode?


Correct:  pal-m
Predicted:  resolution
Question:  In addition to the NTSC system, what 525-line system is considered a standard definition television system?


Correct:  185 million
Predicted:  SES's Astra 1H satellite
Question:  According to the 2010 Satellite Monitor market survey, how many HD capable TV's were sold in Europe?


Correct:  euro1080
Predicted:  kick-start HDTV interest
Question:  Which European company launched HD1?


Correct:  japan
Predicted:  satellite
Question:  Which country had successful public analog HDTV broadcasting?


Correct:  two
Predicted:  the newer and more efficient H.264/MPEG-4 AVC compression standards
Question:  How many frame/field rates were primarily being used in 1983?


Correct:  seven
Predicted:  these files
Question:  How many broadcasters did Japan have sharing a single channel?


Correct:  analog hd
Predicte

Correct:  1989
Predicted:  GoogleTV Roku boxes and AppleTV or built into `` Smart Televisions '' 
Question:  When did satellite test broadcasts of Hi-Vision start?


Correct:  the soviet union
Predicted:  military command
Question:  Who developed Transformator in 1958?


Correct:  hd1
Predicted:  ATSC
Question:  The New Year's Day broadcast officially launched which channel?


Correct:  1979
Predicted:  the 405-line system
Question:  In what year did NHK first develop consumer HD television with a 5:3 aspect ratio?


Correct:  mbaff
Predicted:  a USB 2.0 interface
Question:  Which encoding contains both progressive and interlaced content?


Correct:  digital video broadcasting
Predicted:  standard definition television systems
Question:  What does DVB stand for?


Correct:  1983
Predicted:  to be inaccurately rechristened 'progressive rechristened 'progressive 
Question:  When was France's 819 line system discontinued?


Correct:  8k
Predicted:  normal television
Question:  What is the

KeyboardInterrupt: 